## Improve the accuracy of a vanila CNN

In [55]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
import torchvision
import torchvision.transforms as v2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [44]:
# Load CIFAR-10 dataset
transform = v2.Compose([
    v2.ToTensor(),
    v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [45]:
# Define the vanilla CNN Model
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)  # Output: 32x32x32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 64x32x32
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 64x16x16
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [62]:
def training_loop(model, criterion, optimizer, epochs=5, scheduler=None):
    # Training loop
    for epoch in tqdm(range(epochs), desc="Epochs"):
        for images, labels in train_loader:
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if scheduler is not None:
            scheduler.step()
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")
        
def testing_loop(model):
    # Evaluate on the test set
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

In [47]:
# Vanilla model
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
training_loop(model, criterion, optimizer)
accuracy = testing_loop(model)
print(f"Test Accuracy: {accuracy}%")

Epochs:  20%|██        | 1/5 [00:49<03:18, 49.58s/it]

Epoch [1/5], Loss: 1.4888


Epochs:  40%|████      | 2/5 [01:37<02:25, 48.36s/it]

Epoch [2/5], Loss: 0.5245


Epochs:  60%|██████    | 3/5 [02:28<01:39, 49.98s/it]

...

Epochs:  80%|████████  | 4/5 [03:21<00:50, 50.99s/it]

Epoch [4/5], Loss: 1.1404


Epochs: 100%|██████████| 5/5 [04:08<00:00, 49.74s/it]

Epoch [5/5], Loss: 0.7273


Test Accuracy: 69.92%


In [50]:
# Inbuilt deeper model
model_resnet = torchvision.models.resnet18()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.01)
training_loop(model_resnet, criterion, optimizer)
print(f"Accuracy: {testing_loop(model_resnet)}")

Epochs:  20%|██        | 1/5 [03:44<14:56, 224.09s/it]

Epoch [1/5], Loss: 7.2777


Epochs:  40%|████      | 2/5 [07:29<11:14, 224.75s/it]

Epoch [2/5], Loss: 7.1987


Epochs:  60%|██████    | 3/5 [11:13<07:28, 224.42s/it]

...

Epochs:  80%|████████  | 4/5 [15:01<03:45, 225.80s/it]

Epoch [4/5], Loss: 7.3958


Epochs: 100%|██████████| 5/5 [18:52<00:00, 226.47s/it]

Epoch [5/5], Loss: 7.3149


Accuracy: 0.02


In [51]:
# Incorporating BatchNorm, Dropout
class CNNModel_BN_DO(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 64x32x32
        self.bn2 = nn.BatchNorm2d(64)
        # self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 64x16x16
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        # self.relu3 = nn.ReLU()
        self.do1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.do1(x)
        x = self.fc2(x)
        return x
        

In [56]:
# CNN with BatchNorm and Dropout
model_BN_DO = CNNModel_BN_DO()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_BN_DO.parameters(), lr=0.001)
training_loop(model_BN_DO, criterion, optimizer, epochs=10)
accuracy = testing_loop(model_BN_DO)
print(f"Test Accuracy: {accuracy}%")

Epochs:  10%|█         | 1/10 [00:52<07:55, 52.85s/it]

Epoch [1/10], Loss: 1.5638


Epochs:  20%|██        | 2/10 [01:46<07:05, 53.15s/it]

Epoch [2/10], Loss: 1.3343


Epochs:  30%|███       | 3/10 [02:39<06:13, 53.33s/it]

...

Epochs:  90%|█████████ | 9/10 [07:58<00:52, 52.92s/it]

Epoch [9/10], Loss: 1.0125


Epochs: 100%|██████████| 10/10 [08:48<00:00, 52.84s/it]

Epoch [10/10], Loss: 0.8911


Test Accuracy: 54.86%


In [74]:
# Incorporating BatchNorm, Dropout with weight initialization and a lr scheduler
class CNNModel_BN_DO_weightInitScheduler(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)  # Output: 64x32x32
        self.bn2 = nn.BatchNorm2d(64)
        # self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # Output: 64x16x16
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        # self.relu3 = nn.ReLU()
        self.do1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 10)
        
        self._initialize_weights()
        
    def _initialize_weights(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
                if module.bias is not None:
                    init.constant(module.bias, 0)
            elif isinstance(module, nn.Linear):
                init.xavier_normal_(module.weight)
                if module.bias is not None:
                    init.constant(module.bias, 0)
                    
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.do1(x)
        x = self.fc2(x)
        return x
        

In [75]:
# Model with BatchNorm and Dropout but with weight initialization and a scheduler
# CNN with BatchNorm and Dropout
model_BN_DO_weightInitScheduler = CNNModel_BN_DO_weightInitScheduler()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_BN_DO_weightInitScheduler.parameters(), lr=0.001)  
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)
training_loop(model_BN_DO_weightInitScheduler, criterion, optimizer, epochs=10, scheduler=scheduler)
accuracy = testing_loop(model_BN_DO_weightInitScheduler)
print(f"Test Accuracy: {accuracy}%")

/var/folders/st/2jlnq0ld019_1m1pwbr32slh0000gn/T/ipykernel_6242/1874389079.py:24: FutureWarning: `nn.init.constant` is now deprecated in favor of `nn.init.constant_`.
  init.constant(module.bias, 0)
/var/folders/st/2jlnq0ld019_1m1pwbr32slh0000gn/T/ipykernel_6242/1874389079.py:28: FutureWarning: `nn.init.constant` is now deprecated in favor of `nn.init.constant_`.
  init.constant(module.bias, 0)
Epochs:  10%|█         | 1/10 [00:52<07:52, 52.51s/it]

Epoch [1/10], Loss: 1.9837


Epochs:  20%|██        | 2/10 [01:47<07:12, 54.06s/it]

Epoch [2/10], Loss: 1.8910


Epochs:  30%|███       | 3/10 [02:47<06:37, 56.84s/it]

Epoch [3/10], Loss: 2.2731


Epochs:  40%|████      | 4/10 [03:40<05:30, 55.10s/it]

Epoch [4/10], Loss: 1.7330
